# Introduction

Aujourd'hui la plupart des sites web utilisent du JavaScript pour rendre le site plus dynamique et agréable, mais aussi pour afficher la donnée. 

Pour ces sites, la première méthode abordée n'est pas efficace, car de nombreuses requêtes sont utilisées pour se connecter, afficher la donnée, naviguer, il faudrait prendre en compte toutes ces requêtes et les assembler pour simuler le comportement du site. 

**C'est le comportement de votre navigateur**, il compile toutes les interactions, les requêtes et génère du HTML permettant d'afficher l'interface graphique d'un site web.

Pour palier ce problème, on peut laisser le navigateur faire son travail et contrôler sa surcouche logicielle. Pour cela, on peut prendre le contrôle d'un navigateur comme Chrome, Edge, Firefox ou Safari depuis un script Python. 

Pour cela, nous allons utiliser un package Python `Selenium` https://selenium-python.readthedocs.io/ qui permet d'instancier un navigateur et de le contrôler. Sélénium est beaucoup utilisé pour générer des tests automatiques de sites web. 

On peut trouver les drivers des différents browser : 
- Chrome:	https://googlechromelabs.github.io/chrome-for-testing/#stable (bien télécharger le "chromedriver" et non pas chrome)
- Firefox:	https://github.com/mozilla/geckodriver/releases
- Safari:	https://webkit.org/blog/6900/webdriver-support-in-safari-10/

**Depuis les dernières versions de Selenium, il n'est plus néccessaires de télécharger les driver des browsers car Selenium les manage**

In [5]:
from selenium import webdriver


In [ ]:
chrome = webdriver.Chrome()

Vous devez voir apparaître une nouvelle instance de votre navigateur.
Chrome vous demande depuis peu de selectionner votre navigateur par défaut. Etant donné que notre but est de tout controller depuis notre code, on peut bypass cette demande en ajoutant des options au webdriver de Chrome

In [ ]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")

chrome = webdriver.Chrome(options=chrome_options)

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Maintenant, vous avez le controlle du navigateur depuis Python. Si vous voulez accéder à une page :

In [ ]:
chrome.get("https://books.toscrape.com/")

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00007FF6FAFB2775+28773]
	(No symbol) [0x00007FF6FAF1AFB0]
	(No symbol) [0x00007FF6FADB552A]
	(No symbol) [0x00007FF6FADB2C81]
	(No symbol) [0x00007FF6FADA3989]
	(No symbol) [0x00007FF6FADA56DF]
	(No symbol) [0x00007FF6FADA3C4F]
	(No symbol) [0x00007FF6FADA34CD]
	(No symbol) [0x00007FF6FADA33EA]
	(No symbol) [0x00007FF6FADA1001]
	(No symbol) [0x00007FF6FADA18CC]
	(No symbol) [0x00007FF6FADB85FA]
	(No symbol) [0x00007FF6FAE4FFAE]
	(No symbol) [0x00007FF6FAE2EF9A]
	(No symbol) [0x00007FF6FAE4F1A4]
	(No symbol) [0x00007FF6FAE2ED43]
	(No symbol) [0x00007FF6FADFA548]
	(No symbol) [0x00007FF6FADFB6B1]
	GetHandleVerifier [0x00007FF6FB2DF45D+3358029]
	GetHandleVerifier [0x00007FF6FB2F430D+3443709]
	GetHandleVerifier [0x00007FF6FB2E83FD+3394797]
	GetHandleVerifier [0x00007FF6FB07929B+842635]
	(No symbol) [0x00007FF6FAF2654F]
	(No symbol) [0x00007FF6FAF21

Allez maintenant voir votre fenêtre. Vous êtes sur un faux site de vente de livre. Ce site est spécialement concu pour être scrapé et comprendre les bases, mais en réalité scraper un site est un peu plus complexe, comme vous allez vous en rendre compte dans la section 2 sur Scrapy.

Vous pouvez maintenant vous déplacer dans la page. Pour cela, il va falloir inspecter le code source de la page.

Il existe de nombreuse méthode pour récupérer les éléments par exemple `find_element` qui permet de récupérer un élément grâce au texte affiché. Ici, si on veut récupérer cliquer sur la section `Travel` pour récupérer les livres de voyages

In [ ]:
from selenium.webdriver.common.by import By
link = chrome.find_element(By.LINK_TEXT, "Travel")
type(link)

selenium.webdriver.remote.webelement.WebElement

Si on veut entrer dans le lien, il suffit d'appeler la méthode `click`.

In [ ]:
link.click()

Vous êtes arrivé sur la page des livres de voyage. Maintenant nous allons récupérer toutes les livres. On remarque que tous les livres ont une class HTML nommée `product_pod`.

In [ ]:
all_widgets = chrome.find_elements(By.CLASS_NAME, "product_pod")
type(all_widgets), len(all_widgets)

(list, 20)

On obtient une liste d'éléments décrits par cette class. Il y en a 11, un pour chaque livre. Pour récupérer le titre du premier livre: 

In [ ]:
first_book = all_widgets[0]
first_book_title = first_book.find_element(By.CSS_SELECTOR, "h3")
first_book_title.text

'A Light in the ...'

Si vous êtes fan de JavaScript vous pouvez même injecter du code JS dans le navigateur depuis Python pour exécuter des opérations complexes. 

In [ ]:
js_script = """
const class_name = 'DSIA'
alert(`Hi from ${class_name}`)
"""

In [ ]:
chrome.execute_script(js_script)

# Exercice

## Exercice 1

Extraire le prix des livres :

In [ ]:
def extract_price(book_web_element):
    price = book_web_element.find_element(By.CLASS_NAME, "product_price")
    price2 = price.find_element(By.CLASS_NAME, "price_color")
    return price2.text

extract_price(all_widgets[0])


'£51.77'

Extraire l'image des livres: 

In [ ]:
def extract_image(book_web_element):
    img_container = book_web_element.find_element(By.CLASS_NAME, "image_container")
    img_tag = img_container.find_element(By.TAG_NAME, "img")
    
    # Return the 'src' attribute of the <img> tag
    return img_tag.get_attribute("src")
extract_image(all_widgets[0])

'https://books.toscrape.com/media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg'

Extraire le nombre d'étoiles d'un livre: 

In [ ]:
def extract_rating(book_web_element):
    etoile = book_web_element.find_element(By.CLASS_NAME, "star-rating")
    return etoile.get_attribute("class").split()[-1]
extract_rating(all_widgets[0])

'Three'

In [ ]:
chrome.close()